# MNIST Digit Addition (Two Digit)
This is a simple extension of MNIST digit addition example where we still compute the addition of 2 numbers but each number has 2 digits. In this extension, each number is represented by 2 MNIST images instead of 1 in the original MNIST digit addition example. 

## Imports

In [8]:
import sys
sys.path.append("../../")
import time

import torch
from torchvision import datasets, transforms

from dlpmln import DeepLPMLN
from digit_network import Net

## DeepLPMLN Program

In [9]:
dprogram = '''
img(i1; i2; i3; i4). 
addition(i1,i2,i3,i4,N) :- digit(i1,0,N1), digit(i2,0,N2), digit(i3,0,N3), digit(i4,0,N4), N=N1*10+N2 + N3*10+N4.
nn(m(X,1), digit, [0,1,2,3,4,5,6,7,8,9]) :- img(X).
'''

## Neural Network Instantiation
- Instantiate neural networks.
- Define nnMapping: a dictionary that maps neural network names (i.e., strings) to the neural network objects (i.e., torch.nn.Module object)
- Define optimizers: a dictionary that specifies the optimizer for each network (we use the Adam optimizer here).

In [10]:
m = Net()
nnMapping = {'m':m}
optimizers = {'m':torch.optim.Adam(m.parameters(), lr=0.001)}

## Create DeepLPMLN Object

In [11]:
dlpmlnObj = DeepLPMLN(dprogram, nnMapping, optimizers)

## Create dataList and obsList for Training
### Create Pytorch Dataset and dataLoaders

In [12]:
kwargs = {'num_workers': 0, 'pin_memory': True} if torch.cuda.is_available() else {} 

train_loader = torch.utils.data.DataLoader(
	datasets.MNIST('../../data/', train=True, download=True,
				   transform=transforms.Compose([
					   transforms.ToTensor(),
					   transforms.Normalize((0.1307,), (0.3081,))
				   ])),
	batch_size=4, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
	datasets.MNIST('../../data/', train=False, transform=transforms.Compose([
					   transforms.ToTensor(),
					   transforms.Normalize((0.1307,), (0.3081,))
				   ])),
	batch_size=1000, shuffle=True, **kwargs)

### Construct dataList and obsList

In [13]:
dataList = []
obsList = []
for batch in train_loader:
	dataList.append({"i1":batch[0][0].view(1, 1, 28, 28), "i2":batch[0][1].view(1, 1, 28, 28),"i3":batch[0][2].view(1, 1, 28, 28),"i4":batch[0][3].view(1, 1, 28, 28)})
	obsList.append(":- not addition(i1, i2, i3, i4, {}).".format( batch[1][0]*10+batch[1][1]+batch[1][2]*10+batch[1][3]))

## Training and Testing

In [14]:
time1 = time.time()
acc_list=dlpmlnObj.learn(dataList=dataList, obsList=obsList, epoch=1)
time2 = time.time()
dlpmlnObj.testNN("m", test_loader)
print("--- train time: %s seconds ---" % (time2 - time1))
print("--- test time: %s seconds ---" % (time.time() - time2)) 

Test Accuracy on NN Only for m: 97%
--- train time: 1353.6150598526 seconds ---
--- test time: 2.12981915473938 seconds ---
